# Figures

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec

%matplotlib inline

plt.rcParams["font.family"] = "DejaVu Sans"

<h2 id="tocheading">Table of Contents</h2>
<div id="toc"></div>

In [ ]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

## Some general stuff

- Axis labels start with a lowercase letter, e.g. _altitude_ not _Altitude_

## Bayes' Theorem Visualization

In [ ]:
import scipy.stats as stats

In [ ]:
x, y = np.meshgrid(np.linspace(-1, 1, 201), np.linspace(-1, 1, 201))
pos = np.dstack([x, y])

n = 81
xx = np.empty([n, n], dtype=float)
yy = np.empty([n, n], dtype=float)
prior = np.empty([n, n], dtype=float)
likelihood = np.empty([n, n], dtype=float)
for i, x in enumerate(np.linspace(-1, 1, n)):
    for j, y in enumerate(np.linspace(-1, 1, n)):
        xx[i,j] = x
        yy[i,j] = y
        prior[i,j] = stats.multivariate_normal([0., 0.], [[0.2, 0.], [0., 0.2]]).pdf([x, y])
        likelihood[i,j] = stats.multivariate_normal([0.1*x - 0.3, -0.3*y], [[0.6, 0.58], [0.58, 0.6]]).pdf([x, y])
posterior = likelihood*prior

prior = prior/np.max(prior)
likelihood = likelihood/np.max(likelihood)
posterior = posterior/np.max(posterior)

In [ ]:
levels = 7
cmap = plt.get_cmap("Greys", levels)
norm = plt.Normalize(0.01, 1)

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(9, 3.2))
ax1.contour(xx, yy, prior, levels, cmap=cmap, norm=norm, zorder=-20)
ax1.contourf(xx, yy, prior, levels, cmap=cmap, norm=norm, zorder=-10)
ax1.set_title("Prior distribution", size=11)
ax2.contour(xx, yy, likelihood, levels, cmap=cmap, norm=norm, zorder=-20)
ax2.contourf(xx, yy, likelihood, levels, cmap=cmap, norm=norm, zorder=-10)
ax2.set_title("Likelihood function", size=11)
ax3.contour(xx, yy, posterior, levels, cmap=cmap, norm=norm, zorder=-20)
ax3.contourf(xx, yy, posterior, levels, cmap=cmap, norm=norm, zorder=-10)
ax3.set_title("Posterior distribution", size=11)
for ax in [ax1, ax2, ax3]:
    ax.set_xticks([0])
    ax.set_xticklabels([])
    ax.set_yticks([0])
    ax.set_yticklabels([])
fig.tight_layout()
#fig.savefig("../tex/figures/bayes_theorem.pdf")

## Bayesian Linear Regression

In [ ]:
from regression import LinearRegression

def flatten(*args):
    return (arg.flatten() for arg in args)

In [ ]:
np.random.seed(2)
cov = 0.2**2
x = np.vstack([
        np.random.normal(1.8, size=[6,1]),
        np.random.normal(-1.8, size=[6,1])
        ])
y = np.sin(x) + np.random.normal(scale=np.sqrt(cov), size=x.shape)
x_ref = np.linspace(-5, 5, 100).reshape(-1, 1)

bases = [
        lambda x: np.hstack([1., x, x**2, x**3, x**4, x**5]),
        lambda x: np.hstack([1., x]),
        lambda x: np.hstack([np.exp(-0.5*(x-μ)**2) for μ in range(-3, 4)])
        ]
alphas = [0.1, 0.1, 1]

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(8, 2.7))
for ax, basis, alpha in zip([ax1, ax2, ax3], bases, alphas):
    l = LinearRegression(basis, β=1/cov, α=alpha)
    l.fit(x, y)
    yy, std = flatten(*l.predict(x_ref, samples="std"))
    ax.plot(x_ref, yy, color="k", linewidth=2, zorder=-10)
    ax.fill_between(x_ref.flatten(), yy-std, yy+std, color="#BBBBBB", zorder=-40)
    ax.fill_between(x_ref.flatten(), yy-std*3, yy+std*3, color="#E0E0E0", zorder=-50)
    ax.scatter(*flatten(x, y), 40, edgecolor="#000000", facecolor="#FFFFFF", linewidth=1.2)
    ax.set_xlim(-5, 5)
    ax.set_ylim(-2, 2)
    ax.set_yticks([-2, -1, 0, 1, 2])
    ax.set_yticklabels(["-2", "-1", "0", "1", "2"])
    ax.set_xlabel("predictor")
    ax.label_outer()
ax1.set_ylabel("target")
fig.tight_layout()
#fig.savefig("../tex/figures/bayesian_regression.pdf")

## Absorption in the Microwave Region

In [ ]:
import mwrt
import formulas as fml

In [ ]:
νs = np.linspace(10, 80, 1000)
T = 273.15
θ = 300/T
p = 850.
rh = 100.
esat = fml.esat(T=T)
e = fml.e(esat=esat, RH=rh)
qliq = 0.0001

ylim = 8.0e-7, 1.0e-2
hatpro_o2 = np.array([22.24, 23.04, 23.84, 25.44, 26.24, 27.84, 31.40])
hatpro_hu = np.array([51.26, 52.28, 53.86, 54.94, 56.66, 57.30, 58.00])

def as_absorp(f):
    def absorp(ν, *args, **kwargs):
        return 4*np.pi*ν*1.0e9/299792458.*np.imag(f(ν, *args, **kwargs))
    return absorp

gas_absorp = as_absorp(mwrt.liebe93.refractivity_gaseous)
h2o_absorp = as_absorp(mwrt.liebe93.refractivity_H2O)
cld_absorp = as_absorp(mwrt.tkc.refractivity_lwc)

α_gas = gas_absorp(νs, θ, p-e, e)
α_h2o = h2o_absorp(νs, θ, p-e, e)
α_dry = α_gas - α_h2o
α_cld = qliq * fml.ρ(T=T, p=p, e=e) * cld_absorp(νs, θ)

α_ho2 = gas_absorp(hatpro_o2, θ, p-e, e) + qliq * fml.ρ(T=T, p=p, e=e) * cld_absorp(hatpro_o2, θ)
α_hhu = gas_absorp(hatpro_hu, θ, p-e, e) + qliq * fml.ρ(T=T, p=p, e=e) * cld_absorp(hatpro_hu, θ)

fig, ax = plt.subplots(1, 1, figsize=(8, 3.5))
ax.semilogy(νs, α_gas + α_cld, color="#000000", linewidth=2.5, zorder=-10, label="total")
ax.semilogy(νs, α_cld, linewidth=1.5, color="#666666", zorder=-50, label="cloud")
ax.semilogy(νs, α_dry, linewidth=1.5, color="#33a02c", zorder=-40, label="dry")
ax.semilogy(νs, α_h2o, linewidth=1.5, color="#1f78b4", zorder=-30, label="H₂O")
ax.scatter(hatpro_o2, α_ho2*1.55, 90, marker="|", zorder=-5, color="#000000")
ax.scatter(hatpro_o2, α_ho2*1.35, 20, marker="v", zorder=-5, color="#000000")
ax.scatter(hatpro_hu, α_hhu*1.55, 90, marker="|", zorder=-5, color="#000000")
ax.scatter(hatpro_hu, α_hhu*1.35, 20, marker="v", zorder=-5, color="#000000")

ax.legend(loc="upper left", ncol=2)
ax.text(24, 1.3e-4, "K band", ha="center", fontsize=12)
ax.text(60, 6.0e-4, "V band", ha="center", fontsize=12)

ax.set_xlabel("frequency [GHz]")
ax.set_ylabel("absorption [1/m]")

ax.set_ylim(*ylim)
ax.set_xlim(min(νs), max(νs))
fig.tight_layout()
#fig.savefig("../tex/figures/absorption.pdf")

## Verification of Gaussian Assumption

In [ ]:
from scipy.stats import norm
from db_tools import read_csv_profiles

In [ ]:
T = read_csv_profiles("../data/unified/T_raso.csv")
qvap = read_csv_profiles("../data/unified/qvap_raso.csv")
qliq = read_csv_profiles("../data/unified/qliq_raso.csv")
q = qvap + qliq
lnq = np.log(q)

level = "z=1366m"

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(7.5, 2.3))

counts, *_ = ax1.hist(T[level].values, bins=np.linspace(250, 300, 15), edgecolor="#666666", color="#BBBBBB", linewidth=1.5)
grd = np.linspace(248, 302, 70)
pdf = norm(*norm.fit(T[level].values)).pdf(grd)
ax1.plot(grd, pdf/pdf.max()*counts.max(), color="k", linewidth=1.5)
ax1.set_xticks([250, 260, 270, 280, 290, 300])
ax1.set_xlim(248, 302)
ax1.set_title("T [K]", loc="right", size=10)

counts, *_ = ax2.hist(q[level].values, bins=np.linspace(0, 0.015, 20), edgecolor="#666666", color="#BBBBBB", linewidth=1.5)
ax2.set_xticks([0., 0.005, 0.01, 0.015])
grd = np.linspace(-0.001, 0.016, 70)
pdf = norm(*norm.fit(q[level].values)).pdf(grd)
ax2.plot(grd, pdf/pdf.max()*counts.max(), color="k", linewidth=1.5)
ax2.set_xlim(-0.001, 0.016)
ax2.set_title("q [kg/kg]", loc="right", size=10)

counts, *_ = ax3.hist(lnq[level].values, bins=np.linspace(-8, -3.5, 20), edgecolor="#666666", color="#BBBBBB", linewidth=1.5)
grd = np.linspace(-8.2, -3.2, 70)
pdf = norm(*norm.fit(lnq[level].values)).pdf(grd)
ax3.plot(grd, pdf/pdf.max()*counts.max(), color="k", linewidth=1.5)
ax3.set_xticks([-8, -7, -6, -5, -4])
ax3.set_xlim(-8.2, -3.2)
ax3.set_title("ln(q)", loc="right", size=10)

for ax in [ax1, ax2, ax3]:
    ax.set_title("1366 m", size=10, loc="left")
    ax.set_yticks([])
ax1.set_ylabel("normalized counts")
fig.tight_layout(pad=0.3)
#fig.savefig("../tex/figures/gauss_verification.pdf")

## Weighting Functions

In [ ]:
from mwrt import MWRTM, LinearInterpolation
from optimal_estimation import VirtualHATPRO
import formulas as fml

In [ ]:
z = np.logspace(np.log10(612), np.log10(12612), 150)
T = 288.15 - z * 0.0065
T[z>11000] = 216.65
p = fml.p(z=z, T=T, q=0, p0=940)
rh = 0.1 + (T-216.65)/(T[0] - 216.65) * 0.7
lnq = np.log(fml.qvap(p=p, T=T, RH=rh))

model_grid = np.logspace(np.log10(612), np.log10(12612), 3000)
itp = LinearInterpolation(source=z, target=model_grid)

### Multiple Frequencies at Zenith

In [ ]:
kband = VirtualHATPRO.absorptions[:7]
vband = VirtualHATPRO.absorptions[7:]
titles = ["V band, temperature",
          "K band, humidity"]
colors = ["#"+c*6 for c in ["0", "2", "4", "6", "8", "A", "B"]]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 3.5))
for ax, title in zip([ax1, ax2], titles):
    faps = vband if title.startswith("V") else kband
    res = MWRTM.simulate_radiometer(itp, faps, angles=[0.], p=p, T=T, lnq=lnq)
    jac = res.dT if title.startswith("V") else res.dlnq
    jac[:,0] *= 2
    jac[:,-1] *= 2
    for row, color in zip(jac, colors):
        ax.plot(row/np.max(jac), (z-612)/1000, color=color, linewidth=2)
    ax.set_xticks([0, 0.2, 0.4, 0.6, 0.8, 1.])
    ax.set_ylim(0,8)
    ax.label_outer()
    ax.set_title(title, loc="right", size=11)
ax1.set_ylabel("height above ground [km]")
ax1.set_xlim(-0.18, 1.05)
fig.tight_layout()
#fig.savefig("../tex/figures/jacobian_frequency.pdf")

### Same Frequency at Elevations

In [ ]:
freqs = [54.94, 58.00]
absorp = [VirtualHATPRO.absorptions[-4], VirtualHATPRO.absorptions[-1]]
angles = [0., 60., 65., 70., 75., 80., 85.]
colors = ["#1f78b4"] + ["#"+c*6 for c in ["0", "2", "4", "6", "8", "A", "B"]]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 3.5))
for ax, freq, fap in zip([ax1, ax2], freqs, absorp):
    model = MWRTM(itp, fap)
    res = model(angles=angles, p=p, T=T, lnq=lnq)
    res.dT[:,0] *= 2
    for row, color in zip(res.dT, colors):
        ax.plot(row/np.max(np.abs(res.dT)), (z-612)/1000, color=color, linewidth=2)
    ax.set_ylim(0, 1.)
    ax.label_outer()
    ax.set_title("{:>5.2f} GHz".format(freq), loc="right", size=11)
ax1.set_ylabel("height above ground [km]")
ax1.set_xlim(0, 0.55)
ax1.set_xticks([0, 0.1, 0.2, 0.3, 0.4, 0.5])
ax2.set_xlim(0, 0.35)
ax2.set_xticks([0, 0.1, 0.2, 0.3])
fig.tight_layout()
#fig.savefig("../tex/figures/jacobian_angle.pdf")

## Model Intercomparison

In [ ]:
from db_tools import read_csv_profiles

In [ ]:
mwrtm = read_csv_profiles("../data/unified/training/TB_mwrtm.csv")
mwrtmfap = read_csv_profiles("../data/unified/training/TB_mwrtm_fap.csv")
monortm = read_csv_profiles("../data/unified/training/TB_monortm.csv")
igmk = read_csv_profiles("../data/unified/training/TB_igmk.csv")
cloudy_raso = read_csv_profiles("../data/unified/training/cloudy_raso.csv")["cloudy"]
cloudy_igmk = read_csv_profiles("../data/unified/training/cloudy_igmk.csv")["cloudy"]

zenith = [col for col in mwrtm.columns if col.endswith("_00.0")]

In [ ]:
data1 = mwrtm.loc[~cloudy_raso,zenith]
data2 = monortm.loc[~cloudy_raso,zenith]
data3 = igmk.loc[~cloudy_igmk,zenith]
data4 = mwrtmfap.loc[~cloudy_raso,zenith]
grid = np.arange(0, data1.shape[1])
mean12 = (data1 - data2).mean().values
mean13 = (data1 - data3).mean().values
mean14 = (data1 - data4).mean().values
mn = np.random.multivariate_normal
std12 = mn(mean=np.zeros_like(grid), cov=(data1 - data2).cov().values, size=1000).std(axis=0)
std13 = mn(mean=np.zeros_like(grid), cov=(data1 - data3).cov().values, size=1000).std(axis=0)
std14 = mn(mean=np.zeros_like(grid), cov=(data1 - data4).cov().values, size=1000).std(axis=0)
freqs = ["{:>5.2f}".format(int(col[3:8])/1000) for col in data1.columns]

gs = gridspec.GridSpec(2, 1, height_ratios=[3, 1]) 
fig = plt.figure(figsize=(7.5, 5.5))
ax1 = fig.add_subplot(gs[0])
ax2 = fig.add_subplot(gs[1])
ax1.bar(grid-0.375, mean12, width=0.25, color="#666666", zorder=-20, label="MWRTM - MonoRTM")
ax1.bar(grid-0.125, mean13, width=0.25, color="#BBBBBB", zorder=-20, label="MWRTM - Rosenkranz")
ax1.bar(grid+0.125, mean14, width=0.25, color="#FFFFFF", zorder=-20, label="MWRTM - MWRTM/FAP")
for ax in [ax1, ax2]:
    ax.set_xticks(grid)
    ax.set_xticklabels(["{:>5.2f}".format(int(col[3:8])/1000) for col in data1.columns], size=9)
    ax.tick_params(bottom="off", top="off")
    ax.set_yticks([-1.5, -1, -0.5, 0, 0.5, 1])
    ax.set_title("1581 clear sky cases", size=10, loc="right")
ax1.hlines(0, -0.5, 13.5, color="#000000", zorder=-50)
ax1.vlines(grid+0.5, -2.6, 2.1, color="#E0E0E0", zorder=-55)
ax1.set_ylim(-1.8, 1.3)
ax1.set_title("mean model differences", size=10, loc="left")
ax1.legend(loc="lower left", fontsize=11);
ax1.set_ylabel("brightness temperature [K]")

ax2.bar(grid-0.375, std12, width=0.25, color="#666666", zorder=-20)
ax2.bar(grid-0.125, std13, width=0.25, color="#BBBBBB", zorder=-20)
ax2.bar(grid+0.125, std14, width=0.25, color="#FFFFFF", zorder=-20)
ax2.set_ylabel("br. temp. [K]")
ax2.set_xlabel("channel frequency [GHz]")
ax2.set_title("root mean square of unbiased model differences", size=10, loc="left")
ax2.set_ylim(0, 0.9)
ax2.vlines(grid+0.5, 0, 0.9, color="#E0E0E0", zorder=-55)

fig.tight_layout()
#fig.savefig("../tex/figures/model_comparison.pdf")

## Model Bias

In [ ]:
from db_tools import read_csv_mean

In [ ]:
mwrtm = read_csv_mean("../data/unified/priors/TB_mwrtm_bias.csv")
mwrtmfap = read_csv_mean("../data/unified/priors/TB_mwrtm_fap_bias.csv")
monortm = read_csv_mean("../data/unified/priors/TB_monortm_bias.csv")

zenith = [col for col in mwrtm.index if col.endswith("_00.0")]
freqs = ["{:>5.2f}".format(int(col[3:8])/1000) for col in zenith]
grid = np.arange(0, len(zenith))

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(7.5, 3.5))
ax.bar(grid-0.375, monortm.ix[zenith], width=0.25, color="#666666", zorder=-20, label="MonoRTM")
ax.bar(grid-0.125, mwrtm.ix[zenith], width=0.25, color="#BBBBBB", zorder=-20, label="MWRTM")
ax.bar(grid+0.125, mwrtmfap.ix[zenith], width=0.25, color="#FFFFFF", zorder=-20, label="MWRTM/FAP")

ax.set_xticks(grid)
ax.set_xticklabels(["{:>5.2f}".format(int(col[3:8])/1000) for col in zenith], size=9)
ax.tick_params(bottom="off", top="off")
#ax.set_yticks([-1.5, -1, -0.5, 0, 0.5, 1])
ax.set_title("10 clear sky cases", size=10, loc="right")
ax.hlines(0, -0.5, 13.5, color="#000000", zorder=-50)
ax.vlines(grid+0.5, -0.7, 3.7, color="#E0E0E0", zorder=-55)
ax.set_ylim(-0.7, 3.7)
ax.set_title("RTM - HATPRO bias", size=10, loc="left")
ax.legend(loc="upper right", fontsize=11);
ax.set_ylabel("brightness temperature [K]")
ax.set_xlabel("channel frequency [GHz]")

fig.tight_layout()
#fig.savefig("../tex/figures/model_bias.pdf")

## Prior Distributions

In [ ]:
import datetime as dt
from db_tools import iter_profiles, read_csv_covariance, read_csv_mean
from optimal_estimation import rgrid

### COSMO7

In [ ]:
profiles = iter_profiles("../data/unified/priors/<VAR>_cosmo7+00+06_mean.csv")
Tcov = read_csv_covariance("../data/unified/priors/T_cosmo7+00+06_cov.csv")
lnqcov = read_csv_covariance("../data/unified/priors/lnq_cosmo7+00+06_cov.csv")
for valid, df in profiles:
    if valid == dt.datetime(2015, 9, 11, 3, 48):
        break
T = df["T"].values
T_rand = np.random.multivariate_normal(mean=T, cov=Tcov.values, size=1000)
T_std = np.std(T_rand, axis=0)
lnq = df["lnq"].values
lnq_rand = np.random.multivariate_normal(mean=lnq, cov=lnqcov.values, size=1000)
lnq_std = np.std(lnq_rand, axis=0)
z = (rgrid - rgrid[0])/1000

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 3))

ax1.plot(T, z, color="#000000", linewidth=2, label="mean", zorder=-10)
ax1.fill_betweenx(z, T-T_std, T+T_std, color="#BBBBBB", label="1σ", zorder=-20)
ax1.fill_betweenx(z, T-2*T_std, T+2*T_std, color="#E0E0E0", label="2σ", zorder=-30)
ax1.set_ylim(0, 6)
ax1.set_xlim(243, 292)
ax1.set_title("COSMO-7 prior distributions", loc="left", size=11)
ax1.set_ylabel("height above ground [km]")
ax1.set_xlabel("temperature [K]")

ax2.plot(np.exp(lnq), z, color="#000000", linewidth=2, label="mean", zorder=-10)
ax2.fill_betweenx(z, np.exp(lnq-lnq_std), np.exp(lnq+lnq_std), color="#BBBBBB", label="1σ", zorder=-20)
ax2.fill_betweenx(z, np.exp(lnq-2*lnq_std), np.exp(lnq+2*lnq_std), color="#E0E0E0", label="2σ", zorder=-30)
ax2.set_xticks([0, 0.002, 0.004, 0.006, 0.008, 0.01])
ax2.set_xticklabels(["0", "2", "4", "6", "8", "10"])
ax2.set_ylim(0, 6)
ax2.set_xlim(0, 0.0102)
ax2.set_xlabel("total water content [g/kg]")
ax2.set_title(valid.strftime("%Y-%m-%d %H:%M UTC"), loc="right", size=11)
ax2.label_outer()

fig.tight_layout(pad=0.4)
#ig.savefig("../tex/figures/cosmo7_prior.pdf")

### Radiosonde Climatology

In [ ]:
T = read_csv_mean("../data/unified/priors/T_rasoclim_mean.csv").values
Tcov = read_csv_covariance("../data/unified/priors/T_rasoclim_cov.csv").values
lnq = read_csv_mean("../data/unified/priors/lnq_rasoclim_mean.csv").values
lnqcov = read_csv_covariance("../data/unified/priors/lnq_rasoclim_cov.csv").values
T_rand = np.random.multivariate_normal(mean=T, cov=Tcov, size=1000)
T_std = np.std(T_rand, axis=0)
lnq_rand = np.random.multivariate_normal(mean=lnq, cov=lnqcov, size=1000)
lnq_std = np.std(lnq_rand, axis=0)
z = (rgrid - rgrid[0])/1000

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 3))

ax1.plot(T, z, color="#000000", linewidth=2, label="mean", zorder=-10)
ax1.fill_betweenx(z, T-T_std, T+T_std, color="#BBBBBB", label="1σ", zorder=-20)
ax1.fill_betweenx(z, T-2*T_std, T+2*T_std, color="#E0E0E0", label="2σ", zorder=-30)
ax1.set_ylim(0, 12)
ax1.set_xlim(202, 297)
ax1.set_title("Radiosonde climatology", loc="left", size=11)
ax1.set_ylabel("height above ground [km]")
ax1.set_xlabel("temperature [K]")

ax2.plot(np.exp(lnq), z, color="#000000", linewidth=2, label="mean", zorder=-10)
ax2.fill_betweenx(z, np.exp(lnq-lnq_std), np.exp(lnq+lnq_std), color="#BBBBBB", label="1σ", zorder=-20)
ax2.fill_betweenx(z, np.exp(lnq-2*lnq_std), np.exp(lnq+2*lnq_std), color="#E0E0E0", label="2σ", zorder=-30)
ax2.set_xticks([0, 0.002, 0.004, 0.006, 0.008])
ax2.set_xticklabels(["0", "2", "4", "6", "8"])
ax2.set_ylim(0, 12)
ax2.set_xlim(0., 0.0092)
ax2.set_xlabel("total water content [g/kg]")
ax2.set_title("3561 profiles", loc="right", size=11)
ax2.label_outer()

fig.tight_layout(pad=0.4)
#fig.savefig("../tex/figures/raso_prior.pdf")